In [1]:
from modules.util import load_data, load_result_df
from modules.graphutil import generate_row
from modules.GCN import load_model, generate_output
import torch 
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

dataset, data, G, nodes, ddict = load_data('cora')
model = load_model('GCN3.pth', dataset)
model.eval()

test = load_result_df()

#rand = load_result_df(r'data/finalreport.csv')
#greedy = pd.read_csv(r'data/finalreportgreedy.csv', index_col= 0)


In [ ]:
test

In [2]:
generate_row(G, 0.1, 0.1, dataset, model, ddict, nodes)

[0.1,
 0.1,
 'addremove',
 [0.65561694,
  0.7987616,
  0.89,
  0.8057554,
  0.8078994,
  0.7011494,
  0.74396133,
  0.7718777060508728],
 [178, 129, 267, 448, 225, 122, 77],
 [74, 16, 43, 113, 64, 77, 12],
 [113, 49, 23, 103, 43, 27, 41],
 6.025385519890851,
 4.758520667150108,
 0.7388044979285855]

In [ ]:
rand['Budget'] = rand.AddRatio + rand.RemoveRatio
rand = rand[rand.Budget < 0.30]
greedy = greedy[greedy['Budget'] != 0.15]


In [ ]:
df_metric = pd.concat([rand[['Budget', 'PerturbType','DistMetric', 'DegreeMetric']], greedy[['Budget', 'PerturbType','DistMetric', 'DegreeMetric']]])
df_metric

In [ ]:
def makelegend(ax):
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the plot
    label_dict = {'add' : 'Add', 
            'remove' : 'Delete', 
            'addremove' : 'Add/Delete', 
            'greedy' : 'Greedy'}  # Label dictionary to map original labels to custom labels
    new_labels = [label_dict.get(label, label) for label in labels]  # Map labels to custom labels using the dictionary
    ax.legend(handles + [Line2D([0], [0], color='tab:red', linestyle = 'dashed', lw=1)], new_labels + ['baseline'], fontsize = 'small', ncol = 2)  # Set the custom labels to the legend

hue_colors = {'add' : 'tab:blue', 
            'remove' : 'tab:green', 
            'addremove' : 'tab:orange', 
            'greedy' : 'tab:red'} 
hue_order = ['add', 'addremove', 'remove']
def add_grid(ax):
    ax.grid(); 
    ax.set_axisbelow(True);


In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=df_metric, x= 'Budget', y= 'DistMetric', hue = 'PerturbType', showfliers = False, hue_order= hue_order, palette = hue_colors)
makelegend(ax)
plt.xlabel('Budget')
plt.ylabel('Distribution Metric')
add_grid(ax)
plt.axhline(5.51, linestyle = '--', color = 'red')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=df_metric, x= 'Budget', y= 'DegreeMetric', hue = 'PerturbType', showfliers = False, hue_order = hue_order, palette= hue_colors)
makelegend(ax)
plt.xlabel('Budget')
plt.ylabel('Degree Metric')
add_grid(ax)
plt.axhline(4.08, linestyle = '--', color = 'red')
plt.show()

In [ ]:
#rand['PerturbedTestFScores'] = rand['PerturbedTestFScores'].apply(ast.literal_eval)
vis = rand[(rand.Budget == 0.10) & (rand.PerturbType == 'remove')].reset_index(drop  = True)
vis['PerturbedTestFScores'] = vis['PerturbedTestFScores'].apply(lambda row: row[7])


In [ ]:

fig, ax = plt.subplots(figsize=(5, 4))
x = vis['DegreeMetric']
y = vis['PerturbedTestFScores']
sns.scatterplot( x= x, y= y, ax = ax, marker = '^', s = 30, edgecolor = 'black', color = 'tab:green')
# Calculate R2 score
slope, intercept, r_value, pv, se = stats.linregress(x, y)
# Add R2 score to plot
ax.text(0.85, 0.85, f'R Score: {r_value:.2f}', ha='center', va='center', transform=plt.gca().transAxes)

ax.set_ylabel('F Macro')
ax.set_xlabel('Degree Metric')
add_grid(ax)


In [ ]:
g

In [ ]:

fig, ax = plt.subplots(figsize=(5, 4))
x = vis['DistMetric']
y = vis['PerturbedTestFScores']
sns.scatterplot( x= x, y= y, ax = ax, marker = 's', s = 30, edgecolor = 'black', color = 'tab:green')
# Calculate R2 score
slope, intercept, r_value, pv, se = stats.linregress(x, y)
# Add R2 score to plot
ax.text(0.85, 0.90, f'R Score: {r_value:.2f}', ha='center', va='center', transform=plt.gca().transAxes)

ax.set_ylabel('F Macro')
ax.set_xlabel('Degree Metric')
add_grid(ax)

In [ ]:
greedy['EdgeRatio'] = (greedy['add_count'])/(greedy['Budget'] * 5069)


In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=greedy, x= 'Budget', y= 'EdgeRatio', showfliers = False, color = 'tab:red')
plt.xlabel('Budget')
plt.ylabel('Proportion of Added Edges')
add_grid(ax)
plt.show()

In [ ]:
benchmark = pd.read_csv(r'data/benchmark.csv', index_col= 0)

In [ ]:
avg_dist_metric = []
avg_degree_metric= []
sd_dist_metric= []
sd_degree_metric= []
avg_hp = []


for index, row in greedy.iterrows():
    # Extract add_count and delete_count values from the current row
    add_count = row['add_count']
    delete_count = row['remove_count']
    
    # Slice `benchmark` based on add_count and delete_count
    benchmark_slice = benchmark[(benchmark['AddRatio'] == add_count) & (benchmark['RemoveRatio'] == delete_count)]
    
    # Calculate average and standard deviation of 'DistMetric' and 'DegreeMetric' columns
    avg_dist_metric.append(benchmark_slice['DistMetric'].mean())
    avg_degree_metric.append(benchmark_slice['DegreeMetric'].mean())
    avg_hp.append(benchmark_slice['PerturbedHomophily'].mean())
    sd_dist_metric.append(benchmark_slice['DistMetric'].std())
    sd_degree_metric.append(benchmark_slice['DegreeMetric'].std())

In [ ]:
benchmark
cols = ['PerturbedTestFScores', 'TP', 'FN', 'FP']
for col in cols:
    if col in rand.columns:
        benchmark[col] = benchmark[col].apply(ast.literal_eval)
    if col in greedy.columns:
        greedy[col] = greedy[col].apply(ast.literal_eval)

In [ ]:
import pandas as pd

def create_df(random, greedy, val, bench, labelcount):
    n = len(random.loc[0, val])
    outg = pd.DataFrame();outr = pd.DataFrame()

    outg['Budget'] = greedy.Budget
    outg['PerturbType'] = greedy.PerturbType

    outr['Budget'] = random.AddRatio + random.RemoveRatio
    outr['PerturbType'] = random.PerturbType


    classes = [[] for _ in range(n)]

    for c in range(len(random)):
        for i in range(n):
            if val =='PerturbedTestFScores':
                classes[i].append(random.loc[c, val][i] - bench[i])
            else:
                classes[i].append((random.loc[c, val][i] - bench[i]) / labelcount[i])
    for i, c in enumerate(classes):
        outr[i] = c
    
    classes = [[] for _ in range(n)]
    
    for c in range(len(greedy)):
        for i in range(n):
            if val =='PerturbedTestFScores':
                classes[i].append(greedy.loc[c, val][i] - bench[i])
            else:
                classes[i].append((greedy.loc[c, val][i] - bench[i]) / labelcount[i])
                
    for i, c in enumerate(classes):
        outg[i] = c
    
    out = pd.concat([outg, outr], ignore_index=True)

    out = pd.melt(out, id_vars = ['Budget', 'PerturbType'], value_vars=range(n),
                    var_name='class', value_name=val)  
    return out

df_fscore = create_df(benchmark, greedy, 'PerturbedTestFScores', )

In [ ]:
greedy['benchmark_degree_avg'] = avg_degree_metric
greedy['benchmark_dist_avg'] = avg_dist_metric
greedy['benchmark_degree_sd'] = sd_degree_metric
greedy['benchmark_hp'] = avg_hp

In [ ]:
df_dist = pd.melt(greedy, id_vars='Budget', value_vars=['DistMetric', 'benchmark_dist_avg'], var_name='type', value_name='DistMetric')
df_degree = pd.melt(greedy, id_vars='Budget', value_vars=['DegreeMetric', 'benchmark_degree_avg'], var_name='type', value_name='DegreeMetric')
df_hp = pd.melt(greedy, id_vars='Budget', value_vars=['PerturbedHomophily', 'benchmark_hp'], var_name='type', value_name='PerturbedHomophily')

In [ ]:
from matplotlib.lines import Line2D

def makelegend(ax):
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the plot
    label_dict = {'DistMetric' : 'Greedy', 
            'benchmark_degree_avg' : 'Random Benchmark', 
            'benchmark_dist_avg' : 'Random Benchmark', 
            'DegreeMetric' : 'Greedy', 
            'benchmark_hp' : 'Random Benchmark', 
            'PerturbedHomophily' : 'Greedy'}  # Label dictionary to map original labels to custom labels
    new_labels = [label_dict.get(label, label) for label in labels]  # Map labels to custom labels using the dictionary
    ax.legend(handles + [Line2D([0], [0], color='tab:red', linestyle = 'dashed', lw=1)], new_labels + ['baseline'], fontsize = 'small', ncol = 1)  # Set the custom labels to the legend

hue_colors = {'benchmark_degree_avg' : 'tab:pink', 
            'benchmark_dist_avg' : 'tab:pink', 
            'DegreeMetric' : 'tab:red', 
            'DistMetric' : 'tab:red', 
            'PerturbedHomophily' : 'tab:red', 
            'benchmark_hp' : 'tab:pink'} 

def add_grid(ax):
    ax.grid(); 
    ax.set_axisbelow(True);

In [ ]:
df_hp['PerturbedHomophily'] = df_hp['PerturbedHomophily'] - 0.8041

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=df_hp, x= 'Budget', y= 'PerturbedHomophily', hue = 'type', showfliers = False, palette = hue_colors, linewidth= 0.5)
makelegend(ax)
plt.xlabel('Budget')
plt.ylabel('Homophily')
add_grid(ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=df_dist, x= 'Budget', y= 'DistMetric', hue = 'type', showfliers = False, palette = hue_colors)
makelegend(ax)
plt.axhline(5.51, linestyle = '--', color = 'red')
plt.xlabel('Budget')
plt.ylabel('Distribution Metric')
add_grid(ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

sns.boxplot(data=df_degree, x= 'Budget', y= 'DegreeMetric', hue = 'type', showfliers = False, palette = hue_colors)
makelegend(ax)
plt.axhline(4.08, linestyle = '--', color = 'red')
plt.xlabel('Budget')
plt.ylabel('Degree Metric')
add_grid(ax)
plt.show()